Mount drive for access to data.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy as sc
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import cv2
import librosa

# Import sci-kit models
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

## Load Data

In [ ]:
root_path = 'gdrive/Shareddrives/COSC 522/Data/'  #change dir to your project folder

In [ ]:
# Create data lists
samples = []
labels = []
classes = os.listdir(root_path)

for file in os.listdir(root_path):
  print(file)
  for filename in os.listdir(root_path+file):
    dir_name = root_path+file+"/"
    print(filename)
    
    # Define sample rate
    fs = 44100

    fs_data, y = wavfile.read(os.path.join(dir_name, filename))
    if len(y.shape) == 2:
        y = y[:,0]
    print(fs_data)
    print(y[2570])
    print(type(y))

    if fs_data == fs:
        labels.append([file])
        samples.append(y)
    else:
        print('Warning: Data in {} not sampled at 44.1 kHz!'.format(filename))
        print(f'Sampled at {fs_data}')
        # TODO: Down sample data that is not 44.1 kHz (most seem to be 48.0 kHz)
    
    print()
  # break

Dendropsophus bifurcus
D-bifurcus-sc10743.wav


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  from ipykernel import kernelapp as app


44100
-95
<class 'numpy.ndarray'>

LS100536.WAV
44100
15
<class 'numpy.ndarray'>

LS100594.WAV
44100
7713792
<class 'numpy.ndarray'>

Dbifurcus2.WAV
48000
100096
<class 'numpy.ndarray'>
Sampled at 48000

LS100584.WAV
44100
7324416
<class 'numpy.ndarray'>

LS100582.WAV
44100
12234240
<class 'numpy.ndarray'>

Drhodopeplusbifurcus2_Daniel1.WAV
48000
79037440
<class 'numpy.ndarray'>
Sampled at 48000

LS100303 D. bifurcus sc38590mono.WAV
44100
498944
<class 'numpy.ndarray'>

LS100120.WAV
44100
-1221888
<class 'numpy.ndarray'>

D-bifircus-D-parviceps-LS100120.WAV
44100
-1221888
<class 'numpy.ndarray'>

Engystomops petersi
E-petersi-km-40-sc10741.WAV
44100
0
<class 'numpy.ndarray'>

Epetersi3_Daniel.WAV
48000
-601856
<class 'numpy.ndarray'>
Sampled at 48000

Epetersi2_Daniel1.WAV
48000
171103488
<class 'numpy.ndarray'>
Sampled at 48000

E-petersi-sc28417-km22.WAV
44100
-2309
<class 'numpy.ndarray'>

1017-sc28418-E-petersi-km22mono.WAV
44100
305
<class 'numpy.ndarray'>

Epetersi_Daniel1.WAV
48

In [ ]:
print(len(samples))
print(len(labels))

25
25


## Encode labels

In [ ]:
# Reshape classes list
unique_classes = [[c] for c in classes]

# Create encoder, fit, and check encoding
enc = OneHotEncoder()
enc.fit(unique_classes)
enc_labels = enc.transform(labels).toarray()
print('Encoding is:')
print(enc_labels)

Encoding is:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
for sample in samples:
  print(len(sample)/44100)

593.124716553288
313.6319954648526
317.83473922902493
208.70095238095237
409.18204081632655
240.3265306122449
105.79011337868481
105.79011337868481
641.0274829931973
1018.5607256235828
512.5514739229025
368.0304761904762
356.8213378684807
319.9651700680272
504.0065306122449
419.48589569161
227.33519274376417
307.52507936507936
257.4871428571429
203.3603628117914
119.8402947845805
264.05732426303854
80.80410430839002
200.45142857142858
235.82185941043085


## Get Spectrograms

In [ ]:
def get_spectrograms(time_data, FFT_SIZE=1024, fs=44100):
    ## Compute Fast Fourier Transform of data
    # Create FFT lists
    pxx = []
    
    print("Calculating FFTs for data ...")
    for sample in time_data:
        # flatten() is used because some of the data is stereo rather than mono. This results in having
        # two data points for each data point rather than 1, and something like (# data points, 2) vs. (# data points, ).
        # flatten(), flattens out the second dimension, resulting in (2x # data points, )
#             if windows is True:
        _, _, pxx_idx = signal.spectrogram(sample, nperseg=FFT_SIZE, fs=fs, noverlap=FFT_SIZE/2)
        
        pxx.append(pxx_idx)
                            
    print('Calculated all FFTs')

    return pxx.copy()

In [ ]:
specs = get_spectrograms(samples)

Calculating FFTs for data ...
Calculated all FFTs


In [ ]:
print(len(specs))
print(len(specs[0]))
print(len(specs[0][0]))

25
513
51086


## Binning Data

In [ ]:
def get_bins(spec_data, num_freq_bins=5, num_time_bins=5):
    binned_data = []
        
    # Get individual samples
    for s in spec_data:
        #Open CV's resize takes (columns,rows) as the input for desired size
        resized_pxx=cv2.resize(s[:,:],(num_time_bins,num_freq_bins))
        binned_data.append(resized_pxx.flatten())
            
    return binned_data

In [ ]:
binned_data = get_bins(specs)

In [ ]:
b = np.array(binned_data)
print(b.shape)

(25, 25)


## Get Freq-Windows

In [ ]:
def get_freq_windows(time_data, len_window=1, overlap=0, sample_rate=10):
    len_data = time_data.shape[1]
#     print(len_data)
    
    windows = []
    
    # How far the window should shift
    if overlap == 0:
        step = len_window
        
        start = 0
        stop = step
#         print(len_window)
        for n in range(int(len_data/len_window)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
    else:
        step = int(len_window*overlap)
        
        start = 0
        stop = len_window
        
#         print(len_window)
        for n in range(int(len_data/step)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
        
    return windows

In [ ]:
windowed_freq = []
for fft in specs:
    window_freq_data = get_windows(fft, len_window=2, overlap=.5)
    #shape = np.array(window_data).shape
    #window_data = np.array(window_data).reshape(shape[0], shape[1]*shape[2]) 
    print(np.array(window_data).shape)
    windowed_freq.append(window_data)  

(51085, 513, 2)
(27012, 513, 2)
(27374, 513, 2)
(17974, 513, 2)
(35242, 513, 2)
(20698, 513, 2)
(9110, 513, 2)
(9110, 513, 2)
(55211, 513, 2)
(87729, 513, 2)
(44145, 513, 2)
(31697, 513, 2)
(30732, 513, 2)
(27557, 513, 2)
(43409, 513, 2)
(36129, 513, 2)
(19579, 513, 2)
(26486, 513, 2)
(22176, 513, 2)
(17514, 513, 2)
(10320, 513, 2)
(22742, 513, 2)
(6957, 513, 2)
(17263, 513, 2)
(20310, 513, 2)


## Get Time-Windows

In [ ]:
def get_time_windows(time_data, len_window_sec=1, overlap=0):
  len_data = time_data.shape[0]
  # print(len_data)

  len_window = len_window_sec*44100
    
  windows = []
  # How far the window should shift
  if overlap == 0:
      step = len_window
      
      start = 0
      stop = step
      # print(len_window)

      for n in range(int(len_data/len_window)):
          windows.append(time_data[start:stop])
          
          # print(start)
          # print(stop)
          # print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
  else:
      step = int(len_window*overlap)
      
      start = 0
      stop = len_window
      
  #         print(len_window)
      for n in range(int(len_data/step)):
          windows.append(time_data[start:stop])
          
  #             print(start)
  #             print(stop)
  #             print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
        
  return windows

time_windows = []
num_labels = []
for sample in samples:
  new_windows = get_time_windows(samples[0], len_window_sec=5)
  num_labels.append(len(new_windows))
  print(np.array(new_windows).shape)
  time_windows = time_windows + new_windows

(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)
(118, 220500)


In [ ]:
new_labels = []
for label in labels:
  new_labels = new_labels + [label for num in num_labels]

enc_labels = enc.transform(new_labels).toarray()
print('Encoding is:')
print(enc_labels)

Encoding is:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
train, test = cv.split(np.array(X), np.array(y.argmax(axis=1)))

ValueError: ignored

## Feature Extraction

In [ ]:
specs_reshape = []
for i in range(len(specs)):
  shape = np.array(specs[i]).shape
  X = np.array(specs[i]).reshape(shape[0]*shape[1])
  specs_reshape.append(X)
  

In [ ]:
window_numbers = 20 ## need to be determined
domain_fv = []

for i in range(len(specs_reshape)):
  
    # MFCC
    mfccs = librosa.feature.mfcc(y=specs_reshape[i], sr=44100, n_mfcc=20, win_length = int(np.ceil(1024/window_numbers)), hop_length = int(np.ceil(1024/(2*window_numbers))))
    mfccs_mean = np.mean(mfccs.T, axis=0)
    #mfccs_mean=mfccs_mean.reshape(513,20)

    # Spectral Centroid
#     sc = librosa.feature.spectral_centroid(y=spectrograms[i], sr=44100, win_length = int(np.ceil(1024/5)), hop_length = int(np.ceil(1024/10)))
#     reshape_sc=[]
#     for x in sc:
#         for j in x:
#             reshape_sc.append(j)

    #all_features=list(zip(mfccs_mean,reshape_sc))
    domain_fv.append(mfccs_mean)
    

## Train Model

In [ ]:
# Specify which data to use, these are the only parameters that should change, the rest should remain the same.
X = time_windows
y = enc_labels
depth = 5


# # Create our imputer to replace missing values with the mean 
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp = imp.fit(X)
    
# # Impute our data
# X_imp = imp.transform(X)

# Define model
clf = RandomForestClassifier(max_depth=depth, random_state=0)

# Define number of folds
cv = StratifiedKFold(n_splits=10, shuffle=False)

# Split data, train and test model on 10 folds
split = 1
scores = []
confuse_mat = []
for train_index, test_index in cv.split(np.array(X), np.array(y.argmax(axis=1))):
    print(f"Split {split}/10...")
    x_train, y_train = X[train_index], enc_labels[train_index]
    x_test, y_test = X[test_index], enc_labels[test_index]
    
    
    # Fit model and evaluate it
    clf.fit(x_train, y_train)
    scores.append(clf.score(x_test, y_test))
    
    # Construct confusion matrix
    y_predict = clf.predict(x_test)
    confuse_mat.append(confusion_matrix(y_test.argmax(axis=1), y_predict.argmax(axis=1)))
    
    # Iterate
    split = split + 1
    
print("Mean accuracy:" + str(np.mean(scores)))

ValueError: ignored

In [ ]:
# TODO: should probably do kfold but doing this just for testing purposes
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(binned_data, enc_labels, test_size=.3, stratify=enc_labels)

In [ ]:
clf = RandomForestClassifier()

clf.fit(x_train, y_train)

print(clf.score(x_test, y_test))

0.5


## Save Model

In [ ]:
import pickle

pkl_filename = "pickle_model.pkl"

# Note, for now this will only store the file in volitle memory in the notebook
# To save permanently, most add file path to drive
with open(pkl_filename, 'wb') as file:
  pickle.dump(clf, file)

## Load Model

In [ ]:
with open(pkl_filename, 'rb') as file:
  pickle_model = pickle.load(file)

In [ ]:
# Test saved model
print(pickle_model.score(x_test, y_test))

0.5
